In [11]:
def savingMySQL(NumList, TextList): #mySQL에 새 소식 저장하기
    import pymysql
    from openpyxl import Workbook
    from openpyxl import load_workbook
    conn = pymysql.connect(host='localhost',user='root',password='bottomup13579',db='cnunotifier')
    cursor=conn.cursor()

    cursor.execute("truncate cnu_com_notifier")
    try:
        for i in range(len(NumList)):
            with conn.cursor() as cursor:  #with as 문을 사용하면 close 하지 않아도 알아서 파일을close 해줌
                sql = "insert into cnu_com_notifier values ("+NumList[i]+", '"+TextList[i]+ "')"
                cursor.execute(sql)
                conn.commit()
    finally:
        conn.close()
    return

In [3]:
def getNewsNumberList(): #소식 번호만 리스트로 받아오기
    import pymysql
    from openpyxl import Workbook
    from openpyxl import load_workbook
    conn = pymysql.connect(host='localhost',user='root',password='bottomup13579',db='cnunotifier')
    cursor=conn.cursor()
    try:
        with conn.cursor() as cursor:  #with as 문을 사용하면 close 하지 않아도 알아서 파일을close 해줌
            sql = "select number from cnu_com_notifier order by number desc"
            cursor.execute(sql)
            result_list=[]
            for i in range(5):
                result = cursor.fetchone()
                result_list+=list(result)



    finally:
        conn.close()
    return result_list

In [4]:
def printNewNews(num): #num의 개수만큼 새 소식 받아오기

    import pymysql
    from openpyxl import Workbook
    from openpyxl import load_workbook
    conn = pymysql.connect(host='localhost',user='root',password='bottomup13579',db='cnunotifier')
    cursor=conn.cursor()
    try:
        with conn.cursor() as cursor:  #with as 문을 사용하면 close 하지 않아도 알아서 파일을close 해줌
            sql = "select * from cnu_com_notifier order by number desc"
            cursor.execute(sql)
            result_list=[]
            for i in range(num):
                result = cursor.fetchone()
                result_list+=list(result)
            print(result_list)
    finally:
        conn.close()
    return

In [15]:
def crawlingCNUcom():
    from urllib.request import urlopen
    from bs4 import BeautifulSoup
    html=urlopen("https://computer.cnu.ac.kr/computer/notice/bachelor.do?mode=list&&articleLimit=10&article.offset=0")
    bsObj=BeautifulSoup(html.read(),"html.parser")
    Num=bsObj.html.body.tbody.findAll("tr")
    NumList=[]
    TextList=[]
    for line in Num:
        while len(NumList)<5:
            if '공지' in line.find("td",{"class":"b-num-box"}).get_text():
                break
            else:
                NumList+=[line.find("td",{"class":"b-num-box"}).get_text().strip()]
                TextList+=[line.find("div", {"class":"b-title-box"}).find('a').get_text().strip()]
                
                break
    savingMySQL(NumList,TextList)
  
   
    return

In [6]:
def findNumOfNew(NewList,OldList):
    if NewList[0]==OldList[0]:#List1이 새거
        print('새로운 소식이 없습니다')
        return 0
    else:
        News=str((int)(NewList[0])-(int)(OldList[0]))
        print('새로운 소식이 '+News+'개 있습니다.')
        return int(News)

In [ ]:
import time
while True:
    crawlingCNUcom()
    OldList=getNewsNumberList()
    
    time.sleep(10)
    
    crawlingCNUcom()
    NewList=getNewsNumberList()

    
    numOfNewNews=findNumOfNew(NewList,OldList)
    if numOfNewNews is not 0:
        printNewNews(numOfNewNews)
    
    